<a href="https://colab.research.google.com/github/jayadir/DL_Assignment-2/blob/main/Fine_tuning_gpt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!pip install transformers datasets

In [50]:
import kagglehub

path = kagglehub.dataset_download("paultimothymooney/poetry")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/poetry


In [51]:
from datasets import load_dataset
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import tensorflow as tf
data_files = {
    "train": [
"/root/.cache/kagglehub/datasets/paultimothymooney/poetry/versions/16/bieber.txt"
    ]
}
dataset = load_dataset("text", data_files=data_files)

In [52]:
print((dataset["train"][0]))

{'text': 'What do you mean?'}


In [53]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [54]:
def tokenize_function(txt):
    return tokenizer(txt["text"], truncation=True, padding="max_length", max_length=128)

In [55]:
def gen():
    for row in tokenized_dataset["train"]:
        yield (
            {"input_ids": row["input_ids"], "attention_mask": row["attention_mask"]},
            row["input_ids"]
        )

In [56]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tf_dataset = tf.data.Dataset.from_generator(
    gen,
    output_signature=(
        {
            "input_ids": tf.TensorSpec(shape=(128,), dtype=tf.int32),
            "attention_mask": tf.TensorSpec(shape=(128,), dtype=tf.int32),
        },
        tf.TensorSpec(shape=(128,), dtype=tf.int32),
    )
).shuffle(1000).batch(4)

Map:   0%|          | 0/3715 [00:00<?, ? examples/s]

In [57]:
model = TFGPT2LMHeadModel.from_pretrained("gpt2")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5))

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [58]:
model.fit(tf_dataset, epochs=5, verbose=1)

Epoch 1/5
929/929 [==============================] - 203s 178ms/step - loss: 0.2930
Epoch 2/5
929/929 [==============================] - 164s 176ms/step - loss: 0.1972
Epoch 3/5
929/929 [==============================] - 164s 176ms/step - loss: 0.1539
Epoch 4/5
929/929 [==============================] - 164s 176ms/step - loss: 0.1198
Epoch 5/5
929/929 [==============================] - 164s 176ms/step - loss: 0.0971


In [59]:
model.save_pretrained("gpt2-poetry-tf")
tokenizer.save_pretrained("gpt2-poetry-tf")

('gpt2-poetry-tf/tokenizer_config.json',
 'gpt2-poetry-tf/special_tokens_map.json',
 'gpt2-poetry-tf/vocab.json',
 'gpt2-poetry-tf/merges.txt',
 'gpt2-poetry-tf/added_tokens.json')

In [65]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
model = TFGPT2LMHeadModel.from_pretrained("gpt2-poetry-tf")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-poetry-tf")
prompt = "First you wanna go"
input_ids = tokenizer.encode(prompt, return_tensors='tf')

output = model.generate(
    input_ids,
    max_new_tokens=80,
    temperature=1.0,
    top_k=500,
    top_p=0.95,
    do_sample=True,
    no_repeat_ngram_size=3,
    pad_token_id=tokenizer.eos_token_id,
)

generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_poem)


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2-poetry-tf.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


First you wanna go to the left, then you wanna turn right


In [66]:
input_text = "First you're up"
input_ids = tokenizer(input_text, return_tensors="tf")["input_ids"]
output_temp = model.generate(input_ids, max_length=256, do_sample=True, temperature=1.0, top_k=50)
print(tokenizer.decode(output_temp[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


First you're up, then you're down and in between<|endoftext|>
